In [ ]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm    #自訂的HandTrackingModule.py
import os
# 01. 取得鏡頭影像的基本架構
wCam, hCam = 1280, 720

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
# 02. FPS時間設定
pTime = 0
# 03. 擷取手部資訊
detector = htm.handDetector(detectionCon=0.8, maxHands=1)
# 04. 設定區塊顏色
colorR = (255, 0, 255)
# 05.定義區塊位置與大小
#cx, cy, w, h = 150, 150, 200, 200
# 08. 定義區塊類別
class DragRect():
    def __init__(self, posCenter, size=[200, 200], colorR=[255, 0, 255]):
        self.posCenter = posCenter
        self.size = size
        self.colorR = colorR
    def update(self, cursor, length):
        cx, cy = self.posCenter
        w, h = self.size
        #從下面移上來修改 判斷食指是否點到區塊進行拖曳
        if length < 50:
            if cx-w//2 < cursor[0] < cx+w//2 and cy-h//2 < cursor[1] < cy+h//2:
                self.colorR = (0, 255, 0)
                self.posCenter = cursor
            else:
                self.colorR = (255, 0, 255)    
        else:
            self.colorR = (255, 0, 255)   
# 08. 產生區塊
#rect = DragRect([150, 150])
# 09. 產生更多區塊
rectList = []
for x in range(5):
    rectList.append(DragRect([x*250+150, 150]))
while True:
    success, img = cap.read()
    # 水平翻轉畫面
    img = cv2.flip(img, 1) 
    # 03. 擷取手部資訊
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img)
    # 04. 檢查食指是否移入區塊內
    if len(lmList) != 0: 
        x1, y1 = lmList[8][1:]
        # 07. 使用兩個合併的手指進行拖曳，分開的手指則放開
        # 計算食指和拇指之間的距離
        length, img, lineinfo = detector.findDistance(8, 12, img)
        cv2.putText(img, f'L:{int(length)}', (180, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 3)
        #print(length)
        # 08. 使用區塊類別
        cursor = [x1, y1]
        #rect.update(cursor, length)
        # 09. 檢查更多區塊
        for rect in rectList:
            rect.update(cursor, length)
    # 10. 新增新的畫布
    imgNew = np.zeros_like(img, np.uint8)
        #if length < 50:    # 07. 檢查長度，若合併則拖曳
           # if cx-w//2 < x1 < cx+w//2 and cy-h//2 < y1 < cy+h//2:  # 05. 重新定義區塊位置與大小
               # colorR = (0, 255, 0)
                # 06. 拖曳區塊
               # cx, cy = x1, y1
           # else:
          #      colorR = (255, 0, 255)
      #  else:
         #   colorR = (255, 0, 255)
    # 08. 使用區塊類別更新繪製區塊
    for rect in rectList:     # 09. 繪製更多區塊
        cx, cy = rect.posCenter
        w, h = rect.size
        cv2.rectangle(imgNew, (cx-w//2, cy-h//2), (cx+w//2, cy+h//2), rect.colorR, cv2.FILLED)
    # 10. 設定透明度
    out = img.copy()
    alpha = 0.1
    mask = imgNew.astype(bool)
    out[mask]= cv2.addWeighted(img, alpha, imgNew, 1-alpha, 0)[mask]
    # 04. 繪製區塊
    #cv2.rectangle(img, (cx-w//2, cy-h//2), (cx+w//2, cy+h//2), colorR, cv2.FILLED)   # 05. 重新定義區塊位置與大小    
    # 02. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)

    cv2.imshow("Image", out)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# ch15

In [1]:
import cv2
import time
import numpy as np
import mediapipe as mp
import math

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 1280, 720

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
# 02. FPS時間設定
pTime = 0
# 03. 定義mediapipe pose變數
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

while True:
    success, img = cap.read()
    # 水平翻轉畫面
    img = cv2.flip(img, 1) 
    # 03. 取得pose資料
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(imgRGB)
        
    # 03. 繪出骨架
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(img, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        # 04. 劃出方格
        lmList = []
        bboxInfo = {}
        for id, lm in enumerate(results.pose_landmarks.landmark):
            h, w, c = img.shape
            cx, cy, cz = int(lm.x * w), int(lm.y * h), int(lm.z * w)
            lmList.append([id, cx, cy, cz])
            
        # Bounding Box
        ad = abs(lmList[12][1] - lmList[11][1]) // 2
        if (lmList[16][1] - ad < lmList[12][1] - ad):
            x1 = lmList[16][1] - ad
        else:
            x1 = lmList[12][1] - ad
                
        if (lmList[15][1] - ad > lmList[11][1] - ad):
            x2 = lmList[15][1] + ad
        else:
            x2 = lmList[11][1] + ad
            
        if (lmList[29][2] + ad > lmList[30][2] + ad):
            y2 = lmList[29][2] + ad
        else:
            y2 = lmList[30][2] + ad
            
        y1 = lmList[1][2] - ad

        bbox = (x1, y1, x2 - x1, y2 - y1)
        cx, cy = bbox[0] + (bbox[2] // 2), \
                 bbox[1] + bbox[3] // 2

        bboxInfo = {"bbox": bbox, "center": (cx, cy)}    
                
        cv2.rectangle(img, bbox, (255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
    # 02. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)


    cv2.imshow("Image", img)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [73]:
import cv2
import time
import numpy as np
import mediapipe as mp
import math
import PoseModule as pd    # 05.匯入姿態偵測器

# 01. 取得鏡頭影像的基本架構
wCam, hCam = 1280, 720

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
# 02. FPS時間設定
pTime = 0
# 05.匯入姿態偵測器
detector = pd.PoseDetector()
# 08. 
count = 0
le= 0
dir = 0
dir1 = 0
color = (255, 0, 255)
per = 0
bar = 0

while True:
    success, img = cap.read()
    # 水平翻轉畫面
    img = cv2.flip(img, 1) 
    # 05.匯入姿態偵測器
    img = detector.findPose(img)
    lmList, bboxInfo = detector.findPosition(img)
    # 07.計算角度
    if len(lmList) != 0:
        #左手臂
        angle = detector.findAngle(img, 12, 14, 16, color=(255,0,0))
        # 08. 換算百分比，右臂最小角度為30，最大角度為150
        per = np.interp(angle, (30, 150), (100, 0))
        angle1 = detector.findAngle(img, 11, 13, 15, color=(0,255,0))
        bar = np.interp(angle1, (30, 150), (100, 650))
    if len(lmList) != 0: 
        # 08. 設定顏色
        color = (91, 101, 227)
        if per == 100:
            color = (166, 145, 56)
            if dir == 0:
                count += 0.5
                dir = 1
        if per == 0:
            color = (166, 145, 56)
            if dir == 1:
                count += 0.5
                dir = 0
    if len(lmList) != 0: 
        # 08. 設定顏色
        color = (91, 101, 227)
        if bar == 100:
            color = (166, 145, 56)
            if dir == 0:
                le += 0.5
                dir = 1
        if bar == 0:
            color = (166, 145, 56)
            if dir == 1:
                le += 0.5
                dir = 0     
    # 08. 劃出長條圖
    cv2.rectangle(img, (1100, 100), (1175, 650), color, 3)
    cv2.rectangle(img, (1100, int(bar)), (1175, 650), color, cv2.FILLED)
    cv2.putText(img, f'{int(per)}%', (1080, 75), cv2.FONT_HERSHEY_PLAIN, 4, color, 4)
    # 08. 劃出記分板
    if count<10:
        c = 0
    elif count<100:
        c = 1
    else:
        c = 1
    cv2.rectangle(img, (0, 450), (250+c*120, 720), (92, 91, 96), cv2.FILLED)
    cv2.putText(img, str(int(count)), (45, 670), cv2.FONT_HERSHEY_PLAIN, 15,
                (76, 231, 253), 25) 
        # 08. 劃出記分板
    if le < 0:
        a = 0
    elif le < 100:
        a = 1
    else:
        a = 3
    cv2.rectangle(img, (1280, 720), (1020,500), (92, 91, 96), cv2.FILLED)
    cv2.putText(img, str(int(le)), (1150, 650), cv2.FONT_HERSHEY_PLAIN, 15,
                (76, 231, 253), 25) 
    # 02. 顯示FPS (frame rate)
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS:{int(fps)}', (wCam-120, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 3)

    cv2.imshow("Image", img)
    # 01. 按Q離開
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
